In [ ]:
!pip install pyttsx3

In [2]:
import pyttsx3

engine = pyttsx3.init()
voices = engine.getProperty('voices')
for voice in voices:
    print(voice.id)
    engine.setProperty('voice', voice.id)
    engine.say('The quick brown fox jumped over the lazy dog.')
    engine.say('Я у мамы инженер')

engine.save_to_file('Hello World' , 'test.mp3')
engine.runAndWait()

HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_RU-RU_IRINA_11.0
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0


In [3]:
import pyaudio
import wave
import sys

CHUNK = 1024

def play_stream(wave_stream, player):
    '''Takes a `wave.Wave_read` stream and plays'''
    global CHUNK
    print("Playing...", end="")
    out_stream = player.open(
                    output=True,
                    format=player.get_format_from_width(wave_stream.getsampwidth()),
                    channels=wave_stream.getnchannels(),
                    rate=wave_stream.getframerate()
    )
    data = wave_stream.readframes(CHUNK)
    while data:
        out_stream.write(data)
        data = wave_stream.readframes(CHUNK)
    out_stream.stop_stream()
    out_stream.close()
    print("played")

In [4]:
# after you setup credentials, they will be here: https://console.cloud.google.com/apis/credentials
with open("go.gcp.id") as f:
    api_key = f.readline()

In [5]:
# simple docs: https://cloud.google.com/text-to-speech/docs/quickstart-protocol
# when enabling API just create API key

import requests
import base64
import wave

url = "https://texttospeech.googleapis.com/v1beta1/text:synthesize"

request_data = {
    'input':{
        'text': 'Android is a mobile operating system developed by Google.'
    },
    'voice': {
        'languageCode':'en-gb',
        'name':'en-GB-Standard-A',
        'ssmlGender':'FEMALE'
    },
    'audioConfig':{
        'audioEncoding':'MP3'
    }
}

headers = {
    "X-Goog-Api-Key": api_key,
    "Content-Type": "application/json"
}

print("Requesting mp3 FEMALE encoded text...")

resp = requests.post(url, json=request_data, headers=headers)
assert resp.status_code == 200
sound = resp.json()['audioContent']
_bytes = base64.b64decode(sound)
with open('datasets/sound/google.t2s.mp3', 'wb') as f:
    f.write(_bytes)

print("Done!")
    
print("Changing format and gender...")    
request_data['audioConfig']['audioEncoding'] = 'LINEAR16'
# https://cloud.google.com/text-to-speech/docs/voices
request_data['voice']['languageCode'] = 'en-US'
request_data['voice']['name'] = 'en-US-Standard-B'
request_data['voice']['ssmlGender'] = 'MALE'

resp = requests.post(url, json=request_data, headers=headers)
assert resp.status_code == 200
sound = resp.json()['audioContent']
_bytes = base64.b64decode(sound)

print("Done!")

with open('datasets/sound/google.t2s.wav', 'wb') as f:
    f.write(_bytes)
wf = wave.open('datasets/sound/google.t2s.wav', 'rb')
play_stream(wf, pyaudio.PyAudio())

Requesting mp3 FEMALE encoded text...
Done!
Changing format and gender...
Done!
Playing...played


In [6]:
!pip install --upgrade google-cloud-texttospeech

^C


In [ ]:
# or use API
# pip install --upgrade google-cloud-texttospeech
from google.cloud import texttospeech

# setup credentials file first!!!!

# client = texttospeech.TextToSpeechClient()
# synthesis_input = texttospeech.types.SynthesisInput(text="Hello, World!")

print("Genders:", texttospeech.enums.SsmlVoiceGender._member_names_)
# voice = texttospeech.types.VoiceSelectionParams(
#     language_code='en-US',
#     ssml_gender=texttospeech.enums.SsmlVoiceGender.NEUTRAL)

print("Encodings:", texttospeech.enums.AudioEncoding._member_names_)
# audio_config = texttospeech.types.AudioConfig(audio_encoding=texttospeech.enums.AudioEncoding.MP3)
# response = client.synthesize_speech(synthesis_input, voice, audio_config)

# with open('datasets/sound/google.t2s.api.mp3', 'wb') as out:
#     # Write the response to the output file.
#     out.write(response.audio_content)